<a href="https://colab.research.google.com/github/matheus896/crewai-examples/blob/main/Agents_Pesquisador_e_Escritor_de_artigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --q crewai litellm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from crewai import Agent, Task, Crew, LLM
from litellm import completion
import os

# Importando váriavel
os.environ["GROQ_API_KEY"] = "SUA_API"

# Mente do Agente
llm = LLM (
    model = "groq/llama-3.2-90b-text-preview",
    api_key="GROQ_API_KEY",
    temperature = 0.2
    )

In [12]:
# Criação de agentes
# Defina seus agentes e dê a eles role, goal e backstory

planner = Agent(
    role="Planejador de conteúdo",
    goal= "Planeje um conteúdo envolvente e factualmente preciso sobre {topic}",
    backstory="Você está trabalhando no planejamento de um artigo de blog "
        "sobre o tópico: {topic}."
        "Você coleta informações que ajudam o "
        "público a aprender algo "
        "e tomar decisões informadas. "
        "Seu trabalho é a base para "
        "o Redator de Conteúdo escrever um artigo sobre este tópico.",
    allow_delegation=False,
	  verbose=True,
    memory = True,
    llm=llm
)

writer = Agent(
    role="Redator de Conteúdo",
    goal="Escrever um artigo de opinião perspicaz e factualmente preciso "
             "sobre o tópico: {topic}",
    backstory="Você está trabalhando em uma redação "
                     "de um novo artigo de opinião sobre o tópico: {topic}. "
                     "Você baseia sua redação no trabalho do "
                     "Planejador de Conteúdo, que fornece um esboço "
                     "e um contexto relevante sobre o tópico. "
                     "Você segue os principais objetivos e "
                     "a direção do esboço, "
                     "conforme fornecido pelo Planejador de Conteúdo. "
                     "Você também fornece percepções objetivas e imparciais "
                     "e as apoia com informações "
                     "fornecidas pelo Planejador de Conteúdo. "
                     "Você reconhece em seu artigo de opinião "
                     "quando suas declarações são opiniões "
                     "em vez de declarações objetivas.",
    allow_delegation=False,
    verbose=True,
    Memory = True,
    llm=llm
)

editor = Agent(
    role="Editor",
    goal="Editar um determinado post de blog para alinhar com "
             "o estilo de redação da organização.",
    backstory="Você é um editor que recebe um post de blog "
                     "do Redator de Conteúdo. "
                     "Seu objetivo é revisar o post de blog "
                     "para garantir que ele siga as melhores práticas jornalísticas, "
                     "forneça pontos de vista equilibrados "
                     "ao apresentar opiniões ou afirmações, "
                     "e também evite tópicos ou opiniões "
                     "controversos sempre que possível.",
    allow_delegation=False,
    verbose=True,
    memory=True,
    llm=llm
)

# Criação de tarefas
# Defina suas tarefas e forneça a elas uma descrição, expected_output e agente.

plan = Task(
    description=(
        "1. Priorize as tendências mais recentes, os principais players, "
        "e as notícias relevantes sobre {topic}.\n"
        "2. Identifique o público-alvo, considerando "
        "seus interesses e dores.\n"
        "3. Desenvolva um esboço detalhado de conteúdo, incluindo "
        "uma introdução, pontos-chave e uma chamada para ação.\n"
        "4. Inclua palavras-chave de SEO e dados ou fontes relevantes."
    ),
    expected_output="Um documento de plano de conteúdo abrangente "
        "com um esboço, análise do público-alvo, "
        "palavras-chave de SEO e recursos."
        "O texto deve estar em português BR",
    agent=planner,
)

write = Task(
    description=(
       "1. Use o plano de conteúdo para criar um blog post "
        "envolvente sobre {topic}.\n"
        "2. Incorpore palavras-chave de SEO de forma natural.\n"
        "3. As seções/subtítulos devem ter nomes "
        "de forma envolvente.\n"
        "4. Certifique-se de que o post tenha uma "
        "introdução envolvente, um corpo informativo "
        "e uma conclusão resumida.\n"
        "5. Revise para corrigir erros gramaticais e "
        "alinhamento com a voz da marca.\n"
    ),
    expected_output="Um blog post bem escrito "
        "no formato markdown, pronto para publicação, "
        "cada seção deve ter 2 ou 3 parágrafos."
        "O texto deve estar em português BR",
    agent=writer,
    context = [plan]
)

edit = Task(
    description=("Revise o blog post fornecido para "
               "erros gramaticais e "
               "alinhamento com a voz da marca."),
    expected_output="Um blog post bem escrito no formato markdown, "
                   "pronto para publicação, "
                   "cada seção deve ter 2 ou 3 parágrafos."
                   "O texto deve estar em português BR",
    agent=editor,
    context = [write, plan]
)

# Criação da crew
# Crie sua equipe de agentes
# Passe as tarefas a serem executadas por esses agentes.
# Observação: neste exemplo simples, as tarefas serão executadas sequencialmente (ou seja, elas dependem umas das outras), portanto, a ordem da tarefa na lista é importante.
# verbose=True permite que você veja todos os registros da execução.

crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

# Executando a equipe
# Observação: os LLMs podem fornecer saídas diferentes para a mesma entrada

result = crew.kickoff(inputs={"topic": "Inteligencia Artificial Generativa"})

# Exiba os resultados de sua execução como markdown no notebook.

# Exibir o resultado final como Markdown.

# from IPython.display import Markdown
# Markdown(result.raw)

# Agent: Planejador de conteúdo
## Task: 1. Priorize as tendências mais recentes, os principais players, e as notícias relevantes sobre Inteligencia Artificial Generativa.
2. Identifique o público-alvo, considerando seus interesses e dores.
3. Desenvolva um esboço detalhado de conteúdo, incluindo uma introdução, pontos-chave e uma chamada para ação.
4. Inclua palavras-chave de SEO e dados ou fontes relevantes.


# Agent: Planejador de conteúdo
## Final Answer: 
**Plano de Conteúdo: Inteligência Artificial Generativa**

**Análise do Público-Alvo:**

* **Idade:** 25-45 anos
* **Interesses:** Tecnologia, inovação, inteligência artificial, machine learning, dados e análise
* **Dores:** Dificuldade em entender a complexidade da inteligência artificial, falta de conhecimento sobre as aplicações práticas da inteligência artificial generativa, preocupação com a ética e a responsabilidade na utilização da inteligência artificial
* **Objetivos:** Aprender sobre as tendências mais recentes em int

In [13]:
from IPython.display import Markdown
Markdown(result.raw)

**Inteligência Artificial Generativa: O Futuro da Criação**
===========================================================

**Introdução**
------------

A inteligência artificial generativa é uma área em constante evolução, que está revolucionando a forma como criamos e interagimos com conteúdo. Com a capacidade de gerar imagens, texto e áudio de alta qualidade, essa tecnologia está mudando o jogo em setores como a saúde, finanças e marketing. Neste artigo, vamos explorar as tendências mais recentes em inteligência artificial generativa, os principais players e as notícias relevantes sobre o assunto.

**Tendências Mais Recentes**
---------------------------

### Geração de Imagens e Vídeos

A geração de imagens e vídeos é uma das áreas mais emocionantes da inteligência artificial generativa. Com tecnologias como o DeepDream e o StyleGAN, é possível criar imagens e vídeos de alta qualidade que são quase indistinguíveis da realidade. Essa tecnologia tem aplicações em setores como a publicidade, o entretenimento e a educação.

### Geração de Texto

A geração de texto é outra área em que a inteligência artificial generativa está fazendo um grande impacto. Com tecnologias como o GPT-3 e o BERT, é possível criar texto de alta qualidade que é quase indistinguível da escrita humana. Essa tecnologia tem aplicações em setores como a comunicação, a educação e a pesquisa.

### Geração de Áudio

A geração de áudio é uma área que está em constante evolução. Com tecnologias como o WaveNet e o Tacotron, é possível criar áudio de alta qualidade que é quase indistinguível da realidade. Essa tecnologia tem aplicações em setores como a música, o entretenimento e a comunicação.

**Principais Players**
-------------------

### Google

O Google é um dos principais players na área de inteligência artificial generativa. Com tecnologias como o DeepMind e o TensorFlow, o Google está liderando o desenvolvimento de modelos de inteligência artificial generativa que estão sendo utilizados em setores como a saúde, finanças e marketing.

### Microsoft

A Microsoft também é um dos principais players na área de inteligência artificial generativa. Com tecnologias como o Azure Machine Learning e o Cognitive Toolkit, a Microsoft está desenvolvendo modelos de inteligência artificial generativa que estão sendo utilizados em setores como a saúde, finanças e marketing.

### Facebook

O Facebook também está investindo pesadamente na área de inteligência artificial generativa. Com tecnologias como o FAIR e o PyTorch, o Facebook está desenvolvendo modelos de inteligência artificial generativa que estão sendo utilizados em setores como a comunicação, a educação e a pesquisa.

**Notícias Relevantes**
----------------------

### Desenvolvimento de Modelos de Inteligência Artificial Generativa para a Saúde

A inteligência artificial generativa está sendo utilizada para desenvolver modelos de diagnóstico e tratamento de doenças. Com a capacidade de gerar imagens e vídeos de alta qualidade, essa tecnologia está ajudando a melhorar a precisão do diagnóstico e a eficácia do tratamento.

### Uso da Inteligência Artificial Generativa em Setores como a Finanças e o Marketing

A inteligência artificial generativa está sendo utilizada em setores como a finanças e o marketing para desenvolver modelos de previsão e otimização. Com a capacidade de gerar texto e áudio de alta qualidade, essa tecnologia está ajudando a melhorar a eficácia da comunicação e a precisão da previsão.

**Chamada para Ação**
-------------------

A inteligência artificial generativa é uma área em constante evolução, e é importante investir nela para manter a competitividade no mercado. Com a capacidade de gerar imagens, texto e áudio de alta qualidade, essa tecnologia está mudando o jogo em setores como a saúde, finanças e marketing.

**Recursos**
------------

* **Artigos científicos:** "Generative Adversarial Networks" de Ian Goodfellow et al. (2014)
* **Livros:** "Deep Learning" de Ian Goodfellow et al. (2016)
* **Cursos online:** "Inteligência Artificial Generativa" da Coursera
* **Notícias:** "Google lança novo modelo de inteligência artificial generativa" do The Verge (2022)

Esperamos que este artigo tenha sido útil para entender melhor a inteligência artificial generativa e suas aplicações. Lembre-se de que a inteligência artificial generativa é uma área em constante evolução, e é importante investir nela para manter a competitividade no mercado.